In [6]:
import tensorflow as tf
from tensorflow import keras
from keras.regularizers import l2
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Cropping2D, BatchNormalization, Activation, Conv2DTranspose
from keras.models import Model
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score
import matplotlib.pyplot as plt
from glob import glob
import rasterio
import cv2
import os
import random
import segmentation_models as sm
%matplotlib inline

# Input Pipeline

In [62]:
PATCH_SIZE = 256
H = PATCH_SIZE
W = PATCH_SIZE

In [5]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [106]:
def load_dataset(path, split=0.2): 
    images = sorted(glob(os.path.join(path, "images", "*")))
    labels = sorted(glob(os.path.join(path, "labels", "*")))

    split_size = int(split * len(images))
    
    train_x, val_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, val_y = train_test_split(labels, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y),(val_x, val_y), (test_x, test_y)

def read_image(path, lbl=False):
    path = path.decode()
    x = rasterio.open(path) 
    x = np.array([(x.read(1))/255, (x.read(2))/255, (x.read(3))/255], dtype=np.float32).transpose(1,2,0)
    return x

def read_mask(path):
    path = path.decode()
    x = rasterio.open(path)
    x = np.expand_dims(np.array((x.read(1) > 0.5).astype(np.float32), dtype=np.float32), axis=-1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset



In [107]:
(train_x, train_y),(val_x, val_y), (test_x, test_y) = load_dataset('../data_256')

In [108]:
train_dataset = tf_dataset(train_x, train_y, batch=16)
val_dataset = tf_dataset(val_x, val_y, batch=16)
test_dataset = tf_dataset(test_x, test_y, batch=16)

In [111]:
print(f"Train: {len(train_x)}, \nValidation: {len(val_x)}, \nTest: {len(test_x)}")

Train: 843, 
Validation: 281, 
Test: 281


# Model

In [24]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPooling2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, 2, strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    # print(s1.shape, s2.shape, s3.shape, s4.shape)
    # print(p1.shape, p2.shape, p3.shape, p4.shape)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

In [101]:
input_shape = (256, 256, 3)

model = build_unet(input_shape)
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_78 (Conv2D)             (None, 256, 256, 64  1792        ['input_6[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_74 (BatchN  (None, 256, 256, 64  256        ['conv2d_78[0][0]']              
 ormalization)                  )                                                              

# Training

In [102]:
np.random.seed(42)
tf.random.set_seed(42)

# Optimizer Adam with learning rate 0.0001
optim = keras.optimizers.Adam(0.0001)
# ModelCheckpoint for saving weights of the best model
cb = [keras.callbacks.ModelCheckpoint('./best_unet.h5', save_weights_only=True, save_best_only=True, mode='min'),]

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
# Dice loss and focal loss are good when we have class imbalance 
dice_loss = sm.losses.DiceLoss() 
focal_loss = sm.losses.BinaryFocalLoss() 
total_loss = dice_loss + (1 * focal_loss)

# Metrics: intersection over union (IOU), F1 score
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

model.compile(loss=total_loss, 
              optimizer=optim, 
              metrics=metrics,)


In [112]:
history = model.fit(
   train_dataset,
   epochs=10,
   validation_data=val_dataset,
   shuffle=True,
)

Epoch 1/10


c:\Users\Елена\AppData\Local\Programs\Python\Python310\lib\site-packages\rasterio\__init__.py:321: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


 3/53 [>.............................] - ETA: 41:03 - loss: 0.8767 - iou_score: 0.2905 - f1-score: 0.4237